# Renewable energy sources optimization

### Problem formulation

We need to optimize the distribution of the energy load from multiple sources to different consumers.
We need to to minimize the overall cost, while satisfying demand constraints.

### Parameters

- **Energy Sources**: Two sources (e.g., solar and wind), each with different capacities and costs.
- **Consumers**: Two consumers, each with a specific energy demand.
- **Objective**: Minimize the total cost of energy distribution while meeting all consumer demands without exceeding the source capacities.
- **Tools**: Linear programming, Quadratic Programming

- **Sources parameters**:

    | Source            | Capacity (kW) | Cost (€/kW) |
    |-------------------|---------------|-------------|
    | **I** (eg. Solar) | 100           | 0.15        |                    
    | **J**  (eg. Wind) | 150           | 0.20        |                   


- **Consumer Demands**:
    - Consumer A = 90 kW
    - Consumer B = 120 kW

### Formulation

- **Variables**:
    - $x_{i,a}$: Amount of energy transferred from source $i$ to consumer $a$.
- **Objective**
    We can define a linear cost, to be minimized:
    $$ \text{Cost} = cost_I \times (x_{I,A} + x_{I,B}) + cost_J \times(x_{J,A} + x_{J,B}) $$
- **Constraints**
  At any time these constraint must be met:
    1. Total energy supplied to each consumer meets their demand.
        - Demand of Consumer A: $x_{I,A} + x_{J,A} = 90$
        - Demand of Consumer B: $x_{I,B} + x_{J,B} = 120$
    2. The total energy taken from each source does not exceed its capacity.
        - Solar capacity: $x_{I,A} + x_{I,B} \leq 100$
        - Wind capacity: $x_{J,A} + x_{J,B} \leq 150$
    3. All energies must be positive
       - $x_{i,a} \geq 0$ for any $(i, a)$
         

In [1]:
# GLOBAL VARIABLES

In [2]:
from optimization.core.renewables import EnergySource, Consumer, EnergyDistribution

# Create EnergySource instances
solar = EnergySource("Solar", 100, 0.10)
wind = EnergySource("Wind", 150, 0.05)

# Create Consumer instances
consumer_a = Consumer("A", 15)
consumer_b = Consumer("B", 30)
consumer_c = Consumer("C", 45)

# Create the energy distribution system
system = EnergyDistribution()

system.add_source(solar)
system.add_source(wind)

system.add_consumer(consumer_a)
system.add_consumer(consumer_b)
system.add_consumer(consumer_c)

In [10]:
system.consumers

{'A': {'demand': 15}, 'B': {'demand': 30}, 'C': {'demand': 45}}

In [38]:
import numpy as np 

# cols: source, rows: customer

x_array = np.array([
    [5, 10], 
    [15, 15],
    [20, 25],
]) 

x_vector = x_array.flatten()

x_vector[0:2].sum() == consumer_a.demand

True

In [25]:
def cost_function_array(x_array):
    
    # constraint on conusmer_a
    assert x_array[0, :].sum() == consumer_a.demand

    # constraint on conusmer_b
    assert x_array[1, :].sum() == consumer_b.demand

    # constraint on conusmer_c
    assert x_array[2, :].sum() == consumer_c.demand

    # constraint on source_i
    assert x_array[:, 0].sum() <= solar.capacity

    # constraint on source_j
    assert x_array[:, 1].sum() <= wind.capacity
    
    return solar.cost_per_unit * x_array[:, 0].sum() + wind.cost_per_unit * x_array[:, 1].sum()


def cost_function_vector(x_vector):

    # constraint on conusmer_a
    assert x_vector[:2].sum() == consumer_a.demand
    
    # constraint on conusmer_b
    assert x_vector[2:4].sum() == consumer_b.demand
    
    # constraint on conusmer_c
    assert x_vector[4:].sum() == consumer_c.demand

    # constraint on source_i
    assert x_vector[0::2].sum() <= solar.capacity

    # constraint on source_j
    assert x_array[1::2].sum() <= wind.capacity

    return solar.cost_per_unit * x_vector[0::2].sum() + wind.cost_per_unit * x_vector[1::2].sum()

print(cost_function_array(x_array))
print(cost_function_vector(x_vector))
print(cost_function_array(x_array) == cost_function_vector(x_vector))

6.5
6.5
True


In [28]:
import numpy as np
xrange = np.arange(0, max(consumer_a.demand, consumer_b.demand, consumer_c.demand), 0.5)
xrange

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5,  5. ,
        5.5,  6. ,  6.5,  7. ,  7.5,  8. ,  8.5,  9. ,  9.5, 10. , 10.5,
       11. , 11.5, 12. , 12.5, 13. , 13.5, 14. , 14.5, 15. , 15.5, 16. ,
       16.5, 17. , 17.5, 18. , 18.5, 19. , 19.5, 20. , 20.5, 21. , 21.5,
       22. , 22.5, 23. , 23.5, 24. , 24.5, 25. , 25.5, 26. , 26.5, 27. ,
       27.5, 28. , 28.5, 29. , 29.5, 30. , 30.5, 31. , 31.5, 32. , 32.5,
       33. , 33.5, 34. , 34.5, 35. , 35.5, 36. , 36.5, 37. , 37.5, 38. ,
       38.5, 39. , 39.5, 40. , 40.5, 41. , 41.5, 42. , 42.5, 43. , 43.5,
       44. , 44.5])

In [31]:
from itertools import product

def generate_matrix(xrange):    
    for indices in product(*[xrange] * 6):
        yield np.array(indices)

In [ ]:
xgrid = generate_matrix(xrange)

min_cost = 1e9

for x_vector in generate_matrix(xrange):

    x0, x1, x2, x3, x4, x5 = x_vector
    if x0 + x1 == consumer_a.demand and \
       x2 + x3 == consumer_b.demand and \
       x4 + x5 == consumer_c.demand and \
       x0 + x2 + x4 == solar.capacity and \
       x1 + x3 + x5 == wind.capacity:
        
        cost = cost_function_vector(x_vector)

        if cost < min_cost:
            min_cost = cost 
            print(x_vector, min_cost)

In [ ]:
x_array[:, 1].sum() <= wind.capacity # sum first column, need to satisfy wind capacity <= 150
x_vector[1::2].sum() <= wind.capacity

In [ ]:
x_array[0, :].sum() == consumer_a.demand # sum first row, need to satisfy Custumer's A need == 90
x_vector[:2].sum() == consumer_a.demand

In [ ]:
x_array[1, :].sum() == consumer_b.demand # sum second row, need to satisfy Custumer's B need = 120
x_vector[2:].sum() == consumer_b.demand

In [ ]:
from scipy.optimize import minimize

x = [0, 0, 0, 0]
res = minimize(
    lambda x: solar.cost_per_unit * (x[0] + x[2]) + wind.cost_per_unit * ( x[1] + x[3] ), #what we want to minimize
    x, 
    constraints = (
        {'type':'ineq','fun': lambda x: solar.capacity - (x[0] + x[2]) }, # sum first column, need to satisfy solar capacity <= 100
        {'type':'ineq','fun': lambda x: wind.capacity - (x[1] + x[3]) }, # sum first column, need to satisfy wind capacity <= 150
        {'type':'eq','fun': lambda x: x[0] + x[1] - consumer_a.demand }, # sum first row, need to satisfy Custumer's A need == 90
        {'type':'eq','fun': lambda x: x[2] + x[3] - consumer_b.demand } # sum second row, need to satisfy Custumer's B need = 120   
    ),
    bounds = ((0,None),(0,None),(0,None),(0,None)),
    method='SLSQP',options={'disp': True,'maxiter' : 100000})

print(res)
cost_function_vector(res.x)

### TensorFlow Implementation

We will use TensorFlow to approximate a solution via a gradient descent approach, treating constraints as penalty terms in the loss function.

In [ ]:
import tensorflow as tf

# Initialize variables
x_solar_A = tf.Variable(50.0, trainable=True)  
x_solar_B = tf.Variable(50.0, trainable=True)
x_wind_A = tf.Variable(40.0, trainable=True)
x_wind_B = tf.Variable(80.0, trainable=True)

# Define constraints as penalties
def capacity_constraints():
    return [
        100 - (x_solar_A + x_solar_B),  # Solar capacity
        150 - (x_wind_A + x_wind_B),    # Wind capacity
        x_solar_A + x_wind_A - 90,      # Demand of Consumer A
        x_solar_B + x_wind_B - 120     # Demand of Consumer B
    ]

# Cost function
cost = 3*x_solar_A + 3*x_solar_B + 2*x_wind_A + 2*x_wind_B

# Optimizer
optimizer = tf.optimizers.SGD(learning_rate=0.01)

# Training step
def train_step():
    with tf.GradientTape() as tape:
        constraints = capacity_constraints()
        penalty = tf.reduce_sum(tf.square(constraints))  # Squared penalties for constraint violation
        loss = cost + penalty
    gradients = tape.gradient(loss, [x_solar_A, x_solar_B, x_wind_A, x_wind_B])
    optimizer.apply_gradients(zip(gradients, [x_solar_A, x_solar_B, x_wind_A, x_wind_B]))

# Optimization loop
for _ in range(1000):
    train_step()

# Output the results
print(f'x_solar_A: {x_solar_A.numpy()}, x_solar_B: {x_solar_B.numpy()}')
print(f'x_wind_A: {x_wind_A.numpy()}, x_wind_B: {x_wind_B.numpy()}')
